There are two ways to inject code to watch a program run: tracing and profiling. They are similar, but intended for different purposes and so have different constraints. The easiest, but least efficient, way to monitor a program is through a trace hook, which can be used to write a debugger, monitor code coverage, or achieve many other purposes.

The trace hook is modified by passing a callback function to sys.settrace(). The callback will receive three arguments: the stack frame from the code being run, a string naming the type of notification, and an event-specific argument value. the table below lists the seven event types for different levels of information that occur as a program is being executed.



# Tracing Function call

In [1]:
import sys


def trace_calls(frame, event, arg):
    if event != 'call':
        return
    co = frame.f_code
    func_name = co.co_name
    if func_name == 'write':
        # Ignore write() calls from printing
        return
    func_line_no = frame.f_lineno
    func_filename = co.co_filename
    caller = frame.f_back
    caller_line_no = caller.f_lineno
    caller_filename = caller.f_code.co_filename
    print('* Call to', func_name)
    print('*  on line {} of {}'.format(
        func_line_no, func_filename))
    print('*  from line {} of {}'.format(
        caller_line_no, caller_filename))
    return


def b():
    print('inside b()\n')


def a():
    print('inside a()\n')
    b()


sys.settrace(trace_calls)
a()

* Call to __call__
*  on line 132 of /Users/gaufung/anaconda/lib/python3.6/codeop.py
*  from line 2826 of /Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py
* Call to run_code
*  on line 2851 of /Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py
*  from line 2827 of /Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py
* Call to __getattr__
*  on line 125 of /Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/utils/ipstruct.py
*  from line 2879 of /Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py
* Call to __call__
*  on line 139 of /Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/core/hooks.py
*  from line 2879 of /Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py
* Call to pre_run_code_hook
*  on line 204 of /Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/core/hooks.py
*  from line 149 of

# Tracing inside function

In [2]:
import functools
import sys


def trace_lines(frame, event, arg):
    if event != 'line':
        return
    co = frame.f_code
    func_name = co.co_name
    line_no = frame.f_lineno
    print('*  {} line {}'.format(func_name, line_no))


def trace_calls(frame, event, arg, to_be_traced):
    if event != 'call':
        return
    co = frame.f_code
    func_name = co.co_name
    if func_name == 'write':
        # Ignore write() calls from printing
        return
    line_no = frame.f_lineno
    filename = co.co_filename
    print('* Call to {} on line {} of {}'.format(
        func_name, line_no, filename))
    if func_name in to_be_traced:
        # Trace into this function
        return trace_lines
    return


def c(input):
    print('input =', input)
    print('Leaving c()')


def b(arg):
    val = arg * 5
    c(val)
    print('Leaving b()')


def a():
    b(2)
    print('Leaving a()')


tracer = functools.partial(trace_calls, to_be_traced=['b'])
sys.settrace(tracer)
a()

* Call to <listcomp>
*  on line 123 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py
*  from line 123 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py
* Call to _remap_events
*  on line 96 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py
*  from line 123 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py
* Call to null_wrapper
*  on line 271 of /Users/gaufung/anaconda/lib/python3.6/site-packages/tornado/stack_context.py
*  from line 887 of /Users/gaufung/anaconda/lib/python3.6/site-packages/tornado/ioloop.py
* Call to _handle_events
*  on line 427 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py
*  from line 275 of /Users/gaufung/anaconda/lib/python3.6/site-packages/tornado/stack_context.py
* Call to _handle_recv
*  on line 456 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py
*  from line 440 of /Users/ga

# Watching the Stack

In [3]:
import sys


def trace_calls_and_returns(frame, event, arg):
    co = frame.f_code
    func_name = co.co_name
    if func_name == 'write':
        # Ignore write() calls from printing
        return
    line_no = frame.f_lineno
    filename = co.co_filename
    if event == 'call':
        print('* Call to {} on line {} of {}'.format(
            func_name, line_no, filename))
        return trace_calls_and_returns
    elif event == 'return':
        print('* {} => {}'.format(func_name, arg))
    return


def b():
    print('inside b()')
    return 'response_from_b '


def a():
    print('inside a()')
    val = b()
    return val * 2


sys.settrace(trace_calls_and_returns)
a()

* Call to <listcomp> on line 123 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py
* Call to _remap_events on line 96 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py
* Call to null_wrapper on line 271 of /Users/gaufung/anaconda/lib/python3.6/site-packages/tornado/stack_context.py
* Call to _handle_events on line 427 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py
* Call to _handle_recv on line 456 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py
* Call to recv_multipart on line 370 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/sugar/socket.py
* Call to _run_callback on line 407 of /Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py
* Call to __enter__ on line 219 of /Users/gaufung/anaconda/lib/python3.6/site-packages/tornado/stack_context.py
* Call to null_wrapper on line 271 of /Users/gaufung/anaconda/lib/python3.6/site-

'response_from_b response_from_b '

* schedule => None
* Call to __init__ on line 498 of /Users/gaufung/anaconda/lib/python3.6/threading.py
* Call to __init__ on line 215 of /Users/gaufung/anaconda/lib/python3.6/threading.py
* __init__ => None
* __init__ => None
* Call to schedule on line 180 of /Users/gaufung/anaconda/lib/python3.6/site-packages/ipykernel/iostream.py
* Call to is_alive on line 1104 of /Users/gaufung/anaconda/lib/python3.6/threading.py
* Call to is_set on line 506 of /Users/gaufung/anaconda/lib/python3.6/threading.py
* is_set => True
* Call to _wait_for_tstate_lock on line 1062 of /Users/gaufung/anaconda/lib/python3.6/threading.py
* _wait_for_tstate_lock => None
* is_alive => True
* Call to _event_pipe on line 89 of /Users/gaufung/anaconda/lib/python3.6/site-packages/ipykernel/iostream.py
* _event_pipe => <zmq.sugar.socket.Socket object at 0x10328e888>
* schedule => None
* Call to wait on line 533 of /Users/gaufung/anaconda/lib/python3.6/threading.py
* Call to __enter__ on line 239 of /Users/gaufung/anac